# Model Selection and Validation Notebook

In this notebook you will see the code of the Model Selection and Model Validation sessions.

We will use the swiss dataset for explaining this variable selection process. We can import it from a URL as follows:

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

data_url = "/Users/luisinfanten/Desktop/IE/Classes/First-Year/Second-Semester/Simulating and Modelling/Models/Notebooks/Model Selection/swiss.csv"
swiss = pd.read_csv(data_url, index_col=0)

In [2]:
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
rownames,,,,,,
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In previous sessions we have seen the Backwards stepwise selection manually. We can start with the full model and check which variable to eliminate using the p-values of the beta parameters. Let's start creating the full model:

In [3]:
swiss["Infant_Mortality"] = swiss["Infant.Mortality"]

In [4]:
model_1 = smf.ols(formula = "Fertility ~ Agriculture + Examination + Education + Catholic + Infant_Mortality",
                  data = swiss).fit()
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:              Fertility   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.671
Method:                 Least Squares   F-statistic:                     19.76
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           5.59e-10
Time:                        23:23:10   Log-Likelihood:                -156.04
No. Observations:                  47   AIC:                             324.1
Df Residuals:                      41   BIC:                             335.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           66.9152     10.706  

We see that the variable with the greater p-value is "Examination", so we will create a `model_2` eliminating that variable.

In [5]:
model_2 = smf.ols(formula = "Fertility ~ Agriculture + Education + Catholic + Infant_Mortality",
                  data = swiss).fit()
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:              Fertility   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.671
Method:                 Least Squares   F-statistic:                     24.42
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           1.72e-10
Time:                        23:23:10   Log-Likelihood:                -156.62
No. Observations:                  47   AIC:                             323.2
Df Residuals:                      42   BIC:                             332.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           62.1013      9.605  

This could be a good model for us just using the variable selection strategy that we have studied so far. In this notebook we will see several strategies for variable selection, so we can compare them and take better informed decisions.

## ANOVA method

We will compare different models using ANOVA depending on the difference on their F statistic result.

In [6]:
model_0 = smf.ols(formula = "Fertility ~ 1", data = swiss).fit()
print(model_0.summary())

                            OLS Regression Results                            
Dep. Variable:              Fertility   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 28 Apr 2024   Prob (F-statistic):                nan
Time:                        23:23:10   Log-Likelihood:                -184.86
No. Observations:                  47   AIC:                             371.7
Df Residuals:                      46   BIC:                             373.6
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     70.1426      1.822     38.495      0.0

In [7]:
from statsmodels.stats.anova import anova_lm
anova_lm(model_0, model_1)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,46.0,7177.954894,0.0,NaN,NaN,NaN
1,41.0,2105.042930,5.0,5072.911963,19.761059,5.593799e-10


In [8]:
anova_lm(model_2, model_1)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,42.0,2158.069487,0.0,NaN,NaN,NaN
1,41.0,2105.042930,1.0,53.026557,1.0328,0.315462


## Best Subset Regression

There is not a specific function in Python for obtaining the results of a best subset regression, so we have to create a program for performing those operations (this could be a good programming exercise if you want to try by yourselves). Below, you have my solution for this best subset regression function:

In [9]:
import itertools

# Function to fit a model and return the adjusted R-squared
def fit_model_and_calc_aic(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return {"aic" : model.aic, "bic" : model.bic, "r2" : model.rsquared_adj}

def best_subset_reg(df, y, predictors, metric = "aic"):
  # Initialize the best subset and its performance
  best_subset = []
  best_subset_metric = -np.inf
  best_subset_metric_aic = np.inf
  all_subsets = []
  all_models = []

  # Iterate over all possible subsets of predictors
  for subset_size in range(1, len(predictors) + 1):
      for subset in itertools.combinations(predictors, subset_size):
          X = df[list(subset)]
          model = fit_model_and_calc_aic(y, X)
          all_subsets.append(subset)
          all_models.append(model)
          current_metric = model[metric]

          if (metric == "r2") and (current_metric > best_subset_metric):
              best_subset = subset
              best_subset_metric = current_metric
          elif (metric != "r2") and (current_metric < best_subset_metric_aic):
              best_subset = subset
              best_subset_metric_aic = current_metric

  print("Best subset:", best_subset)
  print(metric, ":", best_subset_metric_aic)
  bestsubreg = pd.DataFrame(all_subsets, all_models)
  return(bestsubreg)


With this function we can print an specific statistic that we want to evaluate, but all the results will be stored in the return of the function, so we can visualize every step of the algorithm.

In [26]:
# Target variable
y = swiss['Fertility']

# Define the predictor variables
predictors = ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant_Mortality']

# Use the function
allsub = best_subset_reg(df = swiss, y = y, predictors = predictors, metric = "aic")

Best subset: ('Agriculture', 'Education', 'Catholic', 'Infant_Mortality')
aic : 323.24084406398185


In [11]:
allsub

,0,1,2,3,4
"{'aic': 367.4674850894165, 'bic': 371.16778029283665, 'r2': 0.10521301901476432}",Agriculture,None,None,None,None
"{'aic': 348.35250343910434, 'bic': 352.05279864252446, 'r2': 0.40421256984504805}",Examination,None,None,None,None
"{'aic': 346.4222968131403, 'bic': 350.1225920165604, 'r2': 0.42818488331778837}",Education,None,None,None,None
"{'aic': 362.34789183539874, 'bic': 366.04818703881887, 'r2': 0.19755913498784483}",Catholic,None,None,None,None
"{'aic': 364.76828652308234, 'bic': 368.46858172650246, 'r2': 0.15515268126541448}",Infant_Mortality,None,None,None,None
"{'aic': 349.090630094017, 'bic': 354.64107289914716, 'r2': 0.4068137507049542}",Agriculture,Examination,None,None,None
"{'aic': 347.69130774820235, 'bic': 353.2417505533325, 'r2': 0.42421425928351864}",Agriculture,Education,None,None,None
"{'aic': 362.31218083638737, 'bic': 367.86262364151753, 'r2': 0.21410906779420746}",Agriculture,Catholic,None,None,None
"{'aic': 357.7882796123653, 'bic': 363.33872241749543, 'r2': 0.28622713022628876}",Agriculture,Infant_Mortality,None,None,None
"{'aic': 342.6291542619213, 'bic': 348.1795970670515, 'r2': 0.483006542211746}",Examination,Education,None,None,None


## Stepwise method

Again, there is not an algorithm in statsmodels that allow us to perform the stepwise selection. We have two options, we can do it by hand or using a different library named “mlxtend”. Both options are used in the notebook.

In [12]:
def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features, dtype='float64')
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < significance_level:
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

def backward_elimination(data, target, significance_level=0.05):
    features = data.columns.tolist()
    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if max_p_value >= significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

def bidirectional_stepwise(data, target, significance_level=0.05):
    forward_features = forward_selection(data, target, significance_level)
    return backward_elimination(data[forward_features], target, significance_level)


In [13]:
# Define the target variable and predictors
target = swiss['Fertility']
predictors = swiss.drop(columns='Fertility')

# Perform forward selection
forward_selected_features = forward_selection(predictors, target)
print("Forward Selection: ", forward_selected_features)

# Perform backward elimination
backward_eliminated_features = backward_elimination(predictors, target)
print("Backward Elimination: ", backward_eliminated_features)

# Perform bidirectional stepwise regression
bidirectional_features = bidirectional_stepwise(predictors, target)
print("Bidirectional Stepwise: ", bidirectional_features)


Forward Selection:  ['Education', 'Catholic', 'Infant_Mortality', 'Infant.Mortality', 'Agriculture']
Backward Elimination:  ['Agriculture', 'Education', 'Catholic', 'Infant.Mortality', 'Infant_Mortality']
Bidirectional Stepwise:  ['Education', 'Catholic', 'Infant_Mortality', 'Infant.Mortality', 'Agriculture']


In [14]:
swiss["Infant_Mortality"] = swiss["Infant.Mortality"]

In [15]:
#!pip install --upgrade mlxtend
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [16]:
from sklearn.linear_model import LinearRegression
#from mlxtend.feature_selection import SequentialFeatureSelector as SFS

target = swiss['Fertility']
predictors = swiss.drop(columns='Fertility')

linear_regression = LinearRegression()

# Forward Selection
forward_selector = SFS(linear_regression,
                       k_features="best",
                       forward=True,
                       floating=False,
                       scoring='r2',
                       cv=0)
forward_selector.fit(predictors, target)
forward_selected_features = list(predictors.columns[list(forward_selector.k_feature_idx_)])
print("Forward Selection: ", forward_selected_features)

# Backward Elimination
backward_selector = SFS(linear_regression,
                        k_features="best",
                        forward=False,
                        floating=False,
                        scoring='r2',
                        cv=0)
backward_selector.fit(predictors, target)
backward_eliminated_features = list(predictors.columns[list(backward_selector.k_feature_idx_)])
print("Backward Elimination: ", backward_eliminated_features)


Forward Selection:  ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']
Backward Elimination:  ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality', 'Infant_Mortality']


In [17]:
def summarize_results(selector, method, predictors):
    selected_features = list(predictors.columns[list(selector.k_feature_idx_)])
    print(f"{method} Results:")
    print("Selected features:", selected_features)
    print("Number of features:", selector.k_feature_names_)
    print("R-squared:", selector.k_score_)
    print("\nFeature Selection History:")
    for idx, values in selector.subsets_.items():
        print("Step", idx, ": Features", list(predictors.columns[list(values["feature_idx"])]), "- R-squared:" ,values["avg_score"])

# Summarize Forward Selection results
summarize_results(forward_selector, "Forward Selection", predictors)
print("\n")
# Summarize Backward Elimination results
summarize_results(backward_selector, "Backward Elimination", predictors)


Forward Selection Results:
Selected features: ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality']
Number of features: ('Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality')
R-squared: 0.7067350015927254

Feature Selection History:
Step 1 : Features ['Education'] - R-squared: 0.4406156467239234
Step 2 : Features ['Education', 'Catholic'] - R-squared: 0.574507122652653
Step 3 : Features ['Education', 'Catholic', 'Infant.Mortality'] - R-squared: 0.662543817438168
Step 4 : Features ['Agriculture', 'Education', 'Catholic', 'Infant.Mortality'] - R-squared: 0.699347583077601
Step 5 : Features ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality'] - R-squared: 0.7067350015927254
Step 6 : Features ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant.Mortality', 'Infant_Mortality'] - R-squared: 0.7067350015927254


Backward Elimination Results:
Selected features: ['Agriculture', 'Examination', 'Education', 'Catholic', '

# Model Validation

Using the best model, now we can obtain the validated scores using the methods we have learned at the beginnig of this module: Holdout (train-test), LOOCV and K-Fold.

In [18]:
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.linear_model import LinearRegression

target = swiss['Fertility']
predictors = swiss.drop(columns=['Fertility', 'Examination']) # we eliminate in this case Examination

## Holdout

In [19]:
# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=2023)

# Create a linear regression model and fit it on the training data
lr_model = LinearRegression().fit(X_train, y_train)

# Calculate the R-squared score on the test data
holdout_score = lr_model.score(X_test, y_test)
print(f"Holdout R-squared: {holdout_score}")

Holdout R-squared: 0.7049044833601623


## LOOCV

the R-squared score is not well-defined when there is only one sample in the test set, which is the case with Leave-One-Out Cross-Validation (LOOCV). In this situation, it's better to use a different scoring metric, such as the mean squared error (MSE) or mean absolute error (MAE).

But in order to compare it, I have developed a program that manually calculates the R-squared for each iteration of the LOOCV method, giving the final average R-squared.

In [20]:
# Create a LeaveOneOut object
loo = LeaveOneOut()

# Create a linear regression model
lr_model = LinearRegression()

# Keep track of true and predicted values
y_true = []
y_pred = []

# Iterate through each split in the LOOCV
for train_index, test_index in loo.split(predictors):
    X_train, X_test = predictors.iloc[train_index], predictors.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    # Fit the model on the training data
    lr_model.fit(X_train, y_train)

    # Make predictions on the test data
    predictions = lr_model.predict(X_test)

    # Append the true and predicted values
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Convert the true and predicted values to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute the R-squared manually
loocv_r2 = 1 - ((y_true - y_pred) ** 2).sum() / ((y_true - y_true.mean()) ** 2).sum()
print(f"LOOCV R-squared: {loocv_r2}")


LOOCV R-squared: 0.6203098431383138


## K-Fold

In [21]:
# Create a KFold object with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=2023)

# Create a linear regression model
lr_model = LinearRegression()

# Perform k-Fold Cross-Validation and calculate the R-squared score for each fold
kfold_scores = cross_val_score(lr_model, predictors, target, cv=kf, scoring='r2')

# Calculate the average R-squared score across all k-Fold Cross-Validation iterations
kfold_avg_score = kfold_scores.mean()
print(f"k-Fold Average R-squared: {kfold_avg_score}")

k-Fold Average R-squared: 0.5538814911931456
